In [39]:
# In this file we try to identify the best parameters for the amplicone design tool given known parameters
#%%
from functools import partial
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple
import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
import pandas as pd
from collections import Counter
from tqdm import tqdm
import time
from Bio.SeqUtils import MeltingTemp
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import primer3
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp
from primer3 import calc_heterodimer
from primer3 import bindings

# functions

In [78]:
reference_genome= '/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa'

def calculate_gc_content(sequence):
    """
    Calculate the percentage of G and C nucleotides in a DNA sequence.

    Args:
        sequence (str): DNA sequence string.

    Returns:
        float: Percentage of G and C nucleotides in the sequence.
    """
    gc_count = 0
    total_count = 0

    for nucleotide in sequence:
        if nucleotide.upper() in ['G', 'C']:
            gc_count += 1
        if nucleotide.upper() in ['A', 'T', 'G', 'C']:
            total_count += 1

    gc_percentage = (gc_count / total_count) * 100
    return gc_percentage

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= '/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None

def complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    return "".join(complement[base] for base in seq)

def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)

def find_sequence_location(query_sequence, position=0, fasta_file=reference_genome):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        print("Primer Sequence not found in FASTA file.")
        return 0,0
    
from difflib import SequenceMatcher
def check_similarity(seq1, seq2, threshold=0.3):
    #Check if two sequences look similar based on their similarity score.
    # Parameters:
    # - seq1: First sequence (string)
    # - seq2: Second sequence (string)
    # - threshold: Similarity threshold (float between 0 and 1)
    # Returns:
    # - True if the sequences look similar, False otherwise
    
    similarity_score = SequenceMatcher(None, seq1, seq2).ratio()
    if similarity_score >= threshold:
        return True
    else:
        return False
    
from Bio.SeqUtils import MeltingTemp
def calculate_tm_gc(primer_sequence):
    # Calculate Tm
    tm = MeltingTemp.Tm_GC(primer_sequence)
    # Calculate GC content
    gc_count = primer_sequence.count('G') + primer_sequence.count('C')
    gc_content = (gc_count / len(primer_sequence)) * 100

    return tm, gc_content
# Example usage
primer = 'ATCGATCGATCG'
tm, gc_content = calculate_tm_gc(primer)
print('Tm:', tm)
print('GC content:', gc_content)

Tm: 30.40290207197791
GC content: 50.0


# loading in the data of known primers

In [6]:
known_primers = pd.read_excel('/mnt/storage10/lwang/Projects/Amplicone_design_tool/amplicon_TB_2023_V2.xlsx', sheet_name='primers', header=1)

In [33]:
known_primers = known_primers[['Drug','primer name', 'forward', 'reverse', 'Amplicon size', 'Start', 'End']]
known_primers = known_primers.iloc[:39,:]

In [61]:
known_primers.head()

,Drug,primer name,forward,reverse,Amplicon size,Start,End
0,Fluoroquinolones,gyrB,AGAGTTGGTGCGGCGTAA,GCCACTTGAGTTTGTACAGC,449.0,6511.0,6959.0
1,Fluoroquinolones,gyrA,GTTGACATCGAGCAGGAGAT,AAATCGACTGTCTCCTCGTC,404.0,7353.0,7756.0
2,Rifampicin,rpoB_1,GGTGGCCTGGAAGAGGT,AGAACCCGAACCGCTCG,514.0,760038.0,760551.0
3,Rifampicin,rpoB_2,CCCCGACCAAAGAGTCA,CTCACGTGACAGACCGC,542.0,760645.0,761186.0
4,Rifampicin,rifampicin,TGCTCACCTCGATCCACATC,ATCTTGCGTTTCTGAGCCAC,400.0,762025.0,762424.0


# getting the tm, gc and primer size range of hte known primers

In [35]:
highest_tm = 0
lowest_tm = 100
highest_gc = 0
lowest_gc = 100
longest_primer = 0
shortest_primer = 100

for i, row in known_primers.iterrows():
    try:
        tm, gc_content = calculate_tm_gc(row['forward'])
    except:
        print(row['forward'])
    if tm > highest_tm:
        highest_tm = tm
    if tm < lowest_tm:
        lowest_tm = tm
    if gc_content > highest_gc:
        highest_gc = gc_content
    if gc_content < lowest_gc:
        lowest_gc = gc_content
        
    tm, gc_content = calculate_tm_gc(row['reverse'])
    if tm > highest_tm:
        highest_tm = tm
    if tm < lowest_tm:
        lowest_tm = tm
    if gc_content > highest_gc:
        highest_gc = gc_content
    if gc_content < lowest_gc:
        lowest_gc = gc_content
        
    if len(row['forward']) > longest_primer:
        longest_primer = len(row['forward'])
    if len(row['forward']) < shortest_primer:
        shortest_primer = len(row['forward'])
    if len(row['reverse']) > longest_primer:
        longest_primer = len(row['reverse'])
    if len(row['reverse']) < shortest_primer:
        shortest_primer = len(row['reverse'])

print('highest_tm:', highest_tm)
print('lowest_tm:', lowest_tm)
print('highest_gc:', highest_gc)
print('lowest_gc:', lowest_gc)
print('longest_primer:', longest_primer)
print('shortest_primer:', shortest_primer)

highest_tm: 58.81956873864458
lowest_tm: 47.06956873864456
highest_gc: 68.42105263157895
lowest_gc: 45.0
longest_primer: 24
shortest_primer: 17


# getting primer3 output

In [100]:
print(len(sequence))

78


In [115]:
sequence = extract_sequence_from_fasta(6511-5, 6959, padding = 0, fasta_file= '/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome')

middle = extract_sequence_from_fasta(30000, 30500, padding = 0, fasta_file= '/mnt/storage10/lwang/Projects/Amplicone_design_tool/GCF_000002765.5_GCA_000002765_genomic.fna', sequence_id='NC_004325.2')

# sequence[:250]+'N'*100+sequence[-250:]
# print(sequence)
results = bindings.design_primers(
            seq_args={
                'SEQUENCE_ID': 'primer',

                'SEQUENCE_TEMPLATE': sequence,
                'SEQUENCE_INCLUDED_REGION': [0,len(sequence)],
                # 'SEQUENCE_INCLUDED_REGION': [(250,len(sequence)-250)],
                # 'SEQUENCE_EXCLUDED_REGION': [(200,300)],
                
                # 'SEQUENCE_TEMPLATE': sequence[:250]+middle+sequence[-250:],
                # 'SEQUENCE_INCLUDED_REGION': [250,750],
                # 'SEQUENCE_EXCLUDED_REGION': [250,750],

                # 'SEQUENCE_INCLUDED_REGION': sequence[:251],
            },
            global_args={
                'PRIMER_NUM_RETURN': 600,
                # 'PRIMER_OPT_SIZE': 20,
                # 'PRIMER_PICK_INTERNAL_OLIGO': 0,
                # 'PRIMER_INTERNAL_MAX_SELF_END': 8,
                # 'PRIMER_MIN_THREE_PRIME_DISTANCE':10,
                # 'PRIMER_MIN_FIVE_PRIME_DISTANCE':10,
                'PRIMER_MIN_SIZE': 15,
                'PRIMER_MAX_SIZE': 36,
                'PRIMER_OPT_TM': 50.0,
                'PRIMER_MIN_TM': 0, #47.0,
                'PRIMER_MAX_TM': 100,#59.0,
                'PRIMER_MIN_GC': 0,#45.0,
                'PRIMER_MAX_GC': 100,#79.0,
                # 'PRIMER_MAX_POLY_X': 5,
                # 'PRIMER_INTERNAL_MAX_POLY_X': 5,
                'PRIMER_SALT_MONOVALENT': 50.0,
                'PRIMER_DNA_CONC': 500.0,
                # 'PRIMER_MAX_NS_ACCEPTED': 2,
                # 'PRIMER_MAX_SELF_ANY': 5,
                # 'PRIMER_MAX_SELF_END': 2,
                # 'PRIMER_PAIR_MAX_COMPL_ANY': 5,
                # 'PRIMER_PAIR_MAX_COMPL_END': 2,
                'PRIMER_PRODUCT_SIZE_RANGE': '440-470',
                # 'PRIMER_PRODUCT_SIZE_RANGE': '550-650',
                # 'PRIMER_PRODUCT_SIZE_RANGE': [
                #     # [950,1050]
                #     [len(sequence)-350,len(sequence)-250]
                # ],
            })

print(results['PRIMER_PAIR_EXPLAIN'])

considered 643, unacceptable product size 38, ok 605


'considered 1302, unacceptable product size 1149, ok 153'

In [81]:
known_primers.head(1)

,Drug,primer name,forward,reverse,Amplicon size,Start,End
0,Fluoroquinolones,gyrB,AGAGTTGGTGCGGCGTAA,GCCACTTGAGTTTGTACAGC,449.0,6511.0,6959.0


# checking primer similarity

In [94]:
find_sequence_location('AGAGTTGGTGCGGCGTAA')

(6510, 6528)

In [118]:
# checking primer similarity
query_seq = 'AGAGTTGGTGCGGCGTAA'
threshold = 0.99
primer_loc = [6510, 6928]
for x in results['PRIMER_LEFT']:
    # print(x['SEQUENCE'], x['TM'], x['GC_PERCENT'])
    seq_loc = find_sequence_location(x['SEQUENCE'])
    # if check_similarity(x['SEQUENCE'], query_seq, threshold=threshold) & seq_loc[0]>primer_loc[0]-10 & seq_loc[1]<primer_loc[1]+10:
    if check_similarity(x['SEQUENCE'], query_seq, threshold=threshold):
        # checking to see if the primer is within 10 bp of the known primer and have a similarity of that passses the threshold
        # print(check_similarity(x['SEQUENCE'], query_seq, threshold=threshold))
        print(x['SEQUENCE'], x['TM'], x['GC_PERCENT'])
        print(seq_loc)

AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)
AGAGTTGGTGCGGCGTAA 62.88089938584807 55.55555555555556
(6510, 6528)


In [58]:
check_similarity('ATCGATCGATCG', 'ATCGATCGATCG')

True

In [22]:
# from Bio.Emboss.primer3 import Primer3
def calculate_primer_properties(primer_sequence):
    primer_properties = {}
    # Calculate 3' self-complementarity of an internal oligo
    primer_properties['internal_oligo_complementarity'] = primer3.calcHomodimer(primer_sequence)
    # Calculate the size of consecutive identical base repeats in a primer
    primer_properties['consecutive_identical_bases'] = primer3.calcHairpin(primer_sequence)
    # Calculate the self-complementarity score for any part of a primer
    # primer_properties['any_part_complementarity'] = primer3.calcHeterodimer(primer_sequence)
    # Calculate the self-complementarity score for the 3' end of a primer
    # primer_properties['three_prime_complementarity'] = primer3.calcEndStability(primer_sequence)
    return primer_properties

calculate_primer_properties('ATCGATCGATCG')

{'internal_oligo_complementarity': ThermoResult(structure_found=True, tm=35.91, dg=-10909.11, dh=-86800.00, ds=-244.69),
 'consecutive_identical_bases': ThermoResult(structure_found=True, tm=50.69, dg=-1174.97, dh=-27800.00, ds=-85.85)}